# OHLCV MT5

In [ ]:
# !pip install MetaTrader5

In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd 

# Kết nối tới MetaTrader 5
if not mt5.initialize():
    print("Khởi tạo MT5 không thành công")
    mt5.shutdown()

# Xác định thời gian bắt đầu và kết thúc để lấy dữ liệu
start_date = datetime(2010, 1, 1)
end_date = datetime(2024, 1, 30)

# Lấy dữ liệu OHLC cho cặp tiền EURUSD trong khoảng thời gian đã xác định
ohlc_data = mt5.copy_rates_range("EURUSD", mt5.TIMEFRAME_D1, start_date, end_date)

# Ngắt kết nối với MT5
mt5.shutdown()

# Chuyển dữ liệu nhận được thành DataFrame và hiển thị
ohlc_df = pd.DataFrame(ohlc_data)
ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
print(ohlc_df)

           time     open     high      low    close  tick_volume  spread  \
0    2010-01-04  1.43259  1.44543  1.42569  1.44111        45505      10   
1    2010-01-05  1.44107  1.44830  1.43467  1.43624        46686      10   
2    2010-01-06  1.43632  1.44338  1.42831  1.43998        45435       9   
3    2010-01-07  1.44001  1.44456  1.42994  1.43041        44298      10   
4    2010-01-08  1.43039  1.44263  1.42634  1.44085        34425       9   
...         ...      ...      ...      ...      ...          ...     ...   
3649 2024-01-23  1.08813  1.09154  1.08207  1.08530        72804      15   
3650 2024-01-24  1.08535  1.09315  1.08463  1.08831        84648      15   
3651 2024-01-25  1.08817  1.09011  1.08208  1.08447        87941      15   
3652 2024-01-26  1.08396  1.08848  1.08121  1.08497        73749      15   
3653 2024-01-29  1.08403  1.08493  1.07951  1.08313        62194      15   

      real_volume  
0               0  
1               0  
2               0  
3      

In [4]:
ohlc_df.to_csv('ohlc_df.csv')

# Bai tap: Day OHLC cua MT5 vao Database (Datetime, Open, High, Low, Close, Volume)

In [ ]:
OHLC MT5 => Day vao 6 cot ben duoi
DWH: Datetime, Open, High, Low, Close, Volume

- Lay du lieu
- View len csv/ print 
- Neu ten cot ko phai Datetime => Rename tu ten cot cu thanh Datetime 
- Gan nhu chinh xac Open, High, Low, Close, Volume (doi tu tick_volume thanh Volume)
- Khoi tao ket noi den SQL Server va insert vao 

=> Hieu ve dataframe: Rename field, xuat csv, insert DB

# Lay OHTC theo timeframe

In [17]:
import MetaTrader5 as mt5
import pandas as pd

# Kết nối đến MetaTrader 5 Server
mt5.initialize()

# Chọn công cụ (symbol) và khung thời gian (timeframe)
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M2  # Đối với OHLC, bạn cần chọn một khung thời gian cụ thể

# Lấy dữ liệu OHLC
ohlcv = mt5.copy_rates_from_pos(symbol, timeframe, 0, 99999)  # Lấy 1000 cây nến gần nhất, maximum duoc 100.000 => DWH

# Đóng kết nối với MetaTrader 5 Server
mt5.shutdown()

# Chuyển đổi dữ liệu thành DataFrame
df = pd.DataFrame(ohlcv)
df['time'] = pd.to_datetime(df['time'], unit='s')  # Chuyển đổi thời gian thành định dạng datetime

# In ra dữ liệu OHLC
print(df[['time', 'open', 'high', 'low', 'close', 'tick_volume']])


                     time     open     high      low    close  tick_volume
0     2023-05-22 15:06:00  1.08040  1.08044  1.08025  1.08036          102
1     2023-05-22 15:08:00  1.08037  1.08039  1.08022  1.08031          121
2     2023-05-22 15:10:00  1.08032  1.08032  1.08015  1.08021          103
3     2023-05-22 15:12:00  1.08021  1.08027  1.07993  1.07999          133
4     2023-05-22 15:14:00  1.07998  1.07999  1.07970  1.07970          133
...                   ...      ...      ...      ...      ...          ...
99994 2023-12-01 14:44:00  1.08539  1.08541  1.08508  1.08523          215
99995 2023-12-01 14:46:00  1.08523  1.08532  1.08498  1.08514          271
99996 2023-12-01 14:48:00  1.08514  1.08519  1.08462  1.08497          359
99997 2023-12-01 14:50:00  1.08497  1.08507  1.08425  1.08448          386
99998 2023-12-01 14:52:00  1.08447  1.08462  1.08434  1.08457          206

[99999 rows x 6 columns]


# Viet rename cac cot lai thanh Datetime, Open, High, Low, Close, Volume va day vao Forex_Data trong DWH tren SQL

# Lay theo khung M45

In [3]:
import MetaTrader5 as mt5
import pandas as pd

# Kết nối đến MetaTrader 5 Server
mt5.initialize()

# Chọn công cụ (symbol) và khung thời gian (timeframe)
symbol = "EURUSD"
custom_timeframe = 45  # Đặt một giá trị tự định nghĩa, ví dụ: 45 phút

# Lấy dữ liệu OHLC
ohlcv = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 900)  # Lấy 900 cây nến M1 gần nhất

# Chuyển đổi dữ liệu thành DataFrame
df = pd.DataFrame(ohlcv)

# Chia nhỏ dữ liệu thành các khung thời gian 45 phút
df['custom_timeframe'] = df.index // (custom_timeframe / 1)  # Tính toán số cụm 45 phút

df_grouped = df.groupby('custom_timeframe', as_index=False).agg({
    'time': 'last',
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'tick_volume': 'sum'
})

# Đóng kết nối với MetaTrader 5 Server
mt5.shutdown()

# Chuyển đổi thời gian thành định dạng datetime
df_grouped['time'] = pd.to_datetime(df_grouped['time'], unit='s')

# In ra dữ liệu OHLC
print(df_grouped[['time', 'open', 'high', 'low', 'close', 'tick_volume']])


                  time     open     high      low    close  tick_volume
0  2024-01-30 00:34:00  1.08290  1.08312  1.08284  1.08289           83
1  2024-01-30 00:54:00  1.08289  1.08323  1.08286  1.08321          164
2  2024-01-30 01:14:00  1.08321  1.08351  1.08319  1.08334          197
3  2024-01-30 01:34:00  1.08335  1.08360  1.08334  1.08355          318
4  2024-01-30 01:54:00  1.08356  1.08359  1.08337  1.08338          314
5  2024-01-30 02:14:00  1.08338  1.08358  1.08326  1.08343          604
6  2024-01-30 02:34:00  1.08344  1.08373  1.08339  1.08366          425
7  2024-01-30 02:54:00  1.08367  1.08414  1.08361  1.08407          724
8  2024-01-30 03:14:00  1.08407  1.08413  1.08331  1.08362          775
9  2024-01-30 03:34:00  1.08362  1.08368  1.08319  1.08342          791
10 2024-01-30 03:54:00  1.08343  1.08367  1.08316  1.08352         1155
11 2024-01-30 04:14:00  1.08351  1.08352  1.08294  1.08305          934
12 2024-01-30 04:34:00  1.08305  1.08318  1.08291  1.08300      